# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor

# Reading Data

In [3]:
# Read the data
X = pd.read_csv('./train.csv', index_col='datetime')
X_test = pd.read_csv('./test.csv', index_col='datetime')

In [3]:
# First 5 rows of X
X.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
# First 5 rows of X_test
X_test.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
datetime,,,,,,,,
2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [5]:
cols_with_missing = [col for col in X.columns
                     if X[col].isnull().any()]
cols_with_missing

[]

there is no missing values in any of the variables!

In [4]:
# Assign the variable count to y
y = X['count']
# Remove the variables we do not want to be in X
X.drop(['casual','registered','count'], axis=1, inplace=True)

# Feature Engineering

## Add year, month and hour variables

In [5]:
X['year'] = X.index.str.slice(0, 4).astype(np.int64)
X['month'] = X.index.str.slice(5, 7).astype(np.int64)
X['hour'] = X.index.str.slice(11, 13).astype(np.int64)

X_test['year'] = X_test.index.str.slice(0, 4).astype(np.int64)
X_test['month'] = X_test.index.str.slice(5, 7).astype(np.int64)
X_test['hour'] = X_test.index.str.slice(11, 13).astype(np.int64)

## OneHot Encode the variable season

In [6]:
# Create season dictionary 
season_dictionary ={1 : 'spring', 2 : 'summer', 3 : 'fall', 4: 'winter'} 
  
# Convert season values to words
X['season'] = X['season'].map(season_dictionary) 
X_test['season'] = X_test['season'].map(season_dictionary) 

# OneHot Encode season
X = pd.get_dummies(X, prefix_sep="_", columns=['season'])
X_test = pd.get_dummies(X_test, prefix_sep="_", columns=['season'])
for col in ['season_fall','season_spring','season_summer','season_winter']:
    X[col] = X[col].astype('int64')
    X_test[col] = X_test[col].astype('int64')

In [7]:
X.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,hour,season_fall,season_spring,season_summer,season_winter
datetime,,,,,,,,,,,,,,
2011-01-01 00:00:00,0,0,1,9.84,14.395,81,0.0,2011,1,0,0,1,0,0
2011-01-01 01:00:00,0,0,1,9.02,13.635,80,0.0,2011,1,1,0,1,0,0
2011-01-01 02:00:00,0,0,1,9.02,13.635,80,0.0,2011,1,2,0,1,0,0
2011-01-01 03:00:00,0,0,1,9.84,14.395,75,0.0,2011,1,3,0,1,0,0
2011-01-01 04:00:00,0,0,1,9.84,14.395,75,0.0,2011,1,4,0,1,0,0


# Modeling

## Splitting data

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

## Random Forest

### Tuning a Random Forest Model with CV Grid Search

In [9]:
# Choose the type of classifier. 
rf_model = RandomForestRegressor()

# Choose some parameter combinations to try
rf_parameters = {'n_estimators': [10, 25, 50, 100, 200, 300], 
                 'max_features': ['log2', 'sqrt','auto'],
                 'criterion': ['mse'],
                 'max_depth': [2, 3, 5, 10], 
                 'min_samples_split': [2, 3, 5],
                 'min_samples_leaf': [1,5,8]
                }
              

# Run the grid search
rf_grid_obj = GridSearchCV(rf_model, rf_parameters, cv=3, scoring='neg_mean_squared_log_error')
rf_grid_obj = rf_grid_obj.fit(X_train, y_train)

# Set the rf_model to the best combination of parameters
rf_model = rf_grid_obj.best_estimator_

c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [11]:
rf_model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=3,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [ ]:
# RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
#                       max_features='auto', max_leaf_nodes=None,
#                       min_impurity_decrease=0.0, min_impurity_split=None,
#                       min_samples_leaf=1, min_samples_split=3,
#                       min_weight_fraction_leaf=0.0, n_estimators=300,
#                       n_jobs=None, oob_score=False, random_state=None,
#                       verbose=0, warm_start=False)

### Fitting the best model to the training data and validating

In [10]:
# Fit the best algorithm to the data. 
rf_model.fit(X_train, y_train)

# Get the score of the model
rf_predictions = rf_model.predict(X_valid)
print(np.sqrt(mean_squared_log_error(y_valid, rf_predictions)))

0.3761147135719048


## XGBoost

### Tuning an XGBoost Model with CV Grid Search

In [13]:
# Choose the type of classifier. 
xgb_model = XGBRegressor()

# Choose some parameter combinations to try
xgb_parameters = {'n_estimators': [100, 200, 500, 1000, 5000], 
                  'max_depth':[3, 5, 7, 9, 10, 15]}

# Run the grid search
xgb_grid_obj = GridSearchCV(xgb_model, xgb_parameters, cv=3, scoring='neg_mean_squared_error')
xgb_grid_obj = xgb_grid_obj.fit(X_train, y_train)

# Set the rf_model to the best combination of parameters
xgb_model = xgb_grid_obj.best_estimator_

c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:59:10] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:59:10] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

[13:08:50] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:09:06] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:09:24] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [21]:
xgb_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=1, gamma=0,
#              importance_type='gain', learning_rate=0.1, max_delta_step=0,
#              max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
#              n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
#              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#              silent=None, subsample=1, verbosity=1)

### Fitting the best model to the training data and validating

In [20]:
# Fit the best algorithm to the data. 
xgb_model.fit(X_train, y_train)

# Get the score of the model
xgb_predictions = xgb_model.predict(X_valid)
xgb_predictions = np.clip(xgb_predictions, y_train.min(), y_train.max())
print(np.sqrt(mean_squared_log_error(y_valid, xgb_predictions)))

c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:17:56] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.38330461551385053


## Combining Models

In [25]:
print(np.sqrt(mean_squared_log_error(y_valid, 
                                     np.mean(np.array([rf_predictions, 
                                                       xgb_predictions]), axis=0 ))))

0.34951151462173147


# Make Test Predictions

In [30]:
# Fitting the rf_model to X
rf_model.fit(X, y)
# Fitting the xgb_model to X
xgb_model.fit(X, y)
# Make predictions using the rf_model
rf_preds_test = rf_model.predict(X_test)
# Make predictions using the xgb_model
xgb_preds_test = xgb_model.predict(X_test)
xgb_preds_test = np.clip(xgb_preds_test, y.min(), y.max())
# Combining predictions
preds_test = np.mean(np.array([rf_preds_test, xgb_preds_test]), axis=0 )

c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
c:\users\rabii\appdata\local\programs\python\python37-32\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[13:29:22] WARNING: d:\build\xgboost\xgboost-0.90.git\src\objective\regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'datetime': X_test.index,
                       'count': preds_test})
output.to_csv('submission.csv', index=False)